In [7]:
import os
import json
from eval_matrix import EvalMatrixChoice

In [15]:
def get_ref_pred(factory_dir):
    ref = []
    pred = []
    with open(os.path.join(factory_dir, "generated_predictions.jsonl")) as file:
        for line in file:
            result = json.loads(line)
            ref.append(result['label'].replace("\n ",""))
            pred.append(result['predict'].replace("\n ",""))
    return ref, pred

In [17]:
def bcexam_formatter(factory_dir, target_dir):
    cutoff = 489
    ref, pred = get_ref_pred(factory_dir)
    simple_ref = ref[:cutoff]
    middle_ref = ref[cutoff:]
    simple_pred = pred[:cutoff]
    middle_pred = pred[cutoff:]
    simple_single_ref = []
    simple_multi_ref = []
    middle_single_ref = []
    middle_multi_ref = []
    simple_single_pred = []
    simple_multi_pred = []
    middle_single_pred = []
    middle_multi_pred = []
    
    for i in range(len(simple_ref)):
        if len(simple_ref[i]) == 1:
            simple_single_ref.append(simple_ref[i])
            simple_single_pred.append(simple_pred[i])
        else:
            simple_multi_ref.append(simple_ref[i])
            simple_multi_pred.append(simple_pred[i])
    
    for i in range(len(middle_ref)):
        if len(simple_ref[i]) == 1:
            middle_single_ref.append(middle_ref[i])
            middle_single_pred.append(middle_pred[i])
        else:
            middle_multi_ref.append(middle_ref[i])
            middle_multi_pred.append(middle_pred[i])

    eval_matrix = EvalMatrixChoice()

    result = {
        "simple": eval_matrix.compute(simple_pred, simple_ref),
        "simple_single": eval_matrix.compute(simple_single_pred, simple_single_ref),
        "simple_multi": eval_matrix.compute(simple_multi_pred, simple_multi_ref),
        "middle": eval_matrix.compute(middle_pred, middle_ref),
        "middle_single": eval_matrix.compute(middle_single_pred, middle_single_ref),
        "middle_multi": eval_matrix.compute(middle_multi_pred, middle_multi_ref),
    }

    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    
    with open(os.path.join(target_dir, "result.json"), 'w', encoding='utf-8') as file:
        json.dump(result, file)

In [23]:
bcexam_formatter("/mnt/pvc-data.common/ChenZikang/codes/LLaMA-Factory/saves/Qwen1.5-7B/lora/eval_2024-10-24-07-15-39","/mnt/pvc-data.common/ChenZikang/codes/mm-BRAG/output/exam_graded/qwen-2.5/lora/usmle-2")